<a href="https://colab.research.google.com/github/kenanmorani/Audio_Signal_Processing_For_Music_Applications_idu/blob/main/Pitch%20Extraction/HW3_Audio_Signal_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 Audio Signal Processing for Music Applications course
Izmir Democracy University, Electrical and Electronics Program    
Submitted to Professor Barsi Bozkurt      
Submitted by Kenan MORANI     

# Installing and importing the required libraries

In [9]:
!pip install essentia mir_eval pysoundfile

In [10]:
import os
from essentia import *
from essentia.standard import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import get_window
import soundfile as sf
import urllib.request
from IPython.display import Audio
import mir_eval

In [11]:
#Downloading a standard mir dataset
!pip install mirdata
import mirdata
orchset = mirdata.initialize('orchset')
orchset.download()

INFO: Downloading ['all'] to /root/mir_datasets/orchset
INFO: [all] downloading Orchset_dataset_0.zip
INFO: /root/mir_datasets/orchset/Orchset_dataset_0.zip already exists and will not be downloaded. Rerun with force_overwrite=True to delete this file and force the download.
INFO: /root/mir_datasets/orchset/GT already exists. Run with force_overwrite=True to download from scratch
INFO: /root/mir_datasets/orchset/README.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /root/mir_datasets/orchset/midi already exists. Run with force_overwrite=True to download from scratch
INFO: /root/mir_datasets/orchset/Orchset - Predominant Melodic Instruments.csv already exists. Run with force_overwrite=True to download from scratch
INFO: /root/mir_datasets/orchset/audio already exists. Run with force_overwrite=True to download from scratch


# Exctracting f0s

In [12]:
#Collecting audio file names 
all_files = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.wav'):
            file_name = os.path.join(root,file)
            all_files.append(file_name)

In [13]:
for file in all_files:
 x, fs = sf.read(file)
 x = x / np.max(np.abs(x))#amplitude normalisation
 minF0 = 50 #in Hz
 maxF0 = 2000 #in Hz
 windowSize = 4096
 hopSize = 1024
 w = get_window('blackman', windowSize)
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 numWindows = startIndexes.size
 Audio(x, rate=fs)
 run_predominant_melody = PitchMelodia(guessUnvoiced = True,
                                      frameSize = windowSize,
                                      hopSize = hopSize,
                                      maxFrequency = maxF0,
                                      minFrequency = minF0,
                                      sampleRate = fs);
 f0, confidence = run_predominant_melody(x)

# Using essentia-melody algorithm

In [16]:
#Collecting mel file names for Annotations 
all_midi = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/GT'):
    for file in files:
        if file.endswith('.mel'):
            file_name = os.path.join(root,file)
            all_midi.append(file_name)
for file in all_midi:        
   #Reading ground-truth
   ref_time, ref_freq = mir_eval.io.load_time_series(file)

#Saving estimated values to file
for file in all_files:
 fileNameF0 = file.replace('.wav','.f0_preDomMel.txt')
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 with open(fileNameF0, 'w') as f:
    for k in range(startIndexes.size):
        time = (startIndexes[k] + windowSize / 2) / fs
        f.write(str(time) + '\t' + str(f0[k]) + '\n')

#Collecting text file names 
all_text = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.txt'):
            file_name = os.path.join(root,file)
            all_text.append(file_name)

for file in all_text:
  #Re-reading Essentia estimation results
  est_time, est_freq = mir_eval.io.load_time_series(file) 

In [17]:
for file in files
  #Running evaluation and ptinting
  scores = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
  for score in scores:
    print(score,'\t',scores[score])

Voicing Recall 	 0.7893447642375995
Voicing False Alarm 	 0.7719298245614035
Raw Pitch Accuracy 	 0.004286589099816289
Raw Chroma Accuracy 	 0.07164727495407226
Overall Accuracy 	 0.011834319526627219


# Using crepe algorithm

In [ ]:
!pip install crepe
import crepe

In [37]:
for file in all_files:
  #Running Essentia predominant melody extraction and saving to an output file
    hopSize = 441#10 miliseconds
    windowSize = 4096
    fs = 44100
    x, fs = sf.read(file)
    x = x / np.max(np.abs(x))#amplitude normalisation
    #x = EqualLoudness()(x)
    est_time, est_freq, confidence, activation = crepe.predict(x, fs,viterbi=True)
    #Running evaluation and ptinting
    scores = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)

for score in scores:
    print(score,'\t',scores[score])

75/75 [==============================] - 108s 1s/step
Voicing Recall 	 1.0
Voicing False Alarm 	 1.0
Raw Pitch Accuracy 	 0.011022657685241886
Raw Chroma Accuracy 	 0.09124311083894672
Overall Accuracy 	 0.010650887573964497
